In [1]:
import deep_predictor
from collections.abc import Mapping
import torch
import scanpy as sc
import numpy as np
import pandas as pd

Global seed set to 0
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)


In [2]:
import scipy.stats as ss
import numpy as np
import scipy.stats as ss
sc.set_figure_params(dpi=250, color_map='viridis')  # low dpi (dots per inch) yields small inline figures
import pandas as pd

In [3]:
adata = sc.read("/home/gaurav/Gaurav/Berlin/Deep_Learning/Experiment/PCA_UMAP/VAE_models/VAE_models/normalised_model/farrel_normalised.h5ad")
annotations = pd.read_csv("/home/gaurav/Gaurav/Berlin/Deep_Learning/Experiment/PCA_UMAP/VAE_models/VAE_models/normalised_model/cell_metadata.csv",index_col = ["Unnamed: 0"])
adata.obs["Annotations"] = annotations["Annotations"]

# Pool Prediction 

In [4]:
## Training on Ectoderm and Predict/Test on Mesoendoderm
## Ectoderm data : Starting segment - 78 , Terminal segment - 3,58,5 


In [5]:
# Starting data 
progenitors_data = adata[adata.obs['segment'] == 78]
progenitors_data = progenitors_data[progenitors_data.obs['HPF'] == 5.3]

# Downsampling Starting data
random_index = np.random.choice(np.arange(0,np.shape(progenitors_data)[0]),1500)
downsampled_proge_data = progenitors_data[random_index]

In [6]:
segments = [58,5]
maturecell_data = adata[adata.obs['segment'] == 3]
maturecell_data = maturecell_data[maturecell_data.obs['HPF'] == 12]
random_index = np.random.choice(np.arange(0,np.shape(maturecell_data)[0]),500)
maturecell_data = maturecell_data[random_index]
for segment in segments:
    segment_data = adata[adata.obs['segment'] == segment]
    segment_data = segment_data[segment_data.obs['HPF'] == 12]
    random_index = np.random.choice(np.arange(0,np.shape(segment_data)[0]),500)
    segment_data = segment_data[random_index]
    maturecell_data = maturecell_data.concatenate(segment_data)

/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observatio

In [7]:
training_data = downsampled_proge_data.concatenate(maturecell_data)
training_data.obs["cells"] = "Training data"

/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [8]:
zebra_obj = deep_predictor.DeepPredictor(training_data,latent_dim=100,hidden_layers=[800,800],workers=8,parameters=[0,0,1,0,1],likelihood="nb",batch_size=100)

/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755853668/work/c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


In [9]:
zebra_obj.train(epochs=100)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type     | Params
-----------------------------------
0 | model | VAEmodel | 29.1 M
-----------------------------------
29.1 M    Trainable params
0         Non-trainable params
29.1 M    Total params
116.567   Total estimated model params size (MB)
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:1892: PossibleUserWarning: The number of training batches (30) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


In [10]:
zebra_obj.save_model(path="/home/gaurav/Gaurav/Berlin/Paper/Models_single_vs_pool/Trained_on_mesoendoderm_predict_ectoderm/Pool_meso.pt")

# Single-type Prediction 

In [12]:
## Training on Ectoderm and Predict/Test on Mesoendoderm
## Ectoderm data : Starting segment - 78 , Terminal segment - 3,58,5 

In [11]:
# Starting data 
progenitors_data = adata[adata.obs['segment'] == 78]
progenitors_data = progenitors_data[progenitors_data.obs['HPF'] == 5.3]

# Downsampling Starting data
random_index = np.random.choice(np.arange(0,np.shape(progenitors_data)[0]),500)
downsampled_proge_data = progenitors_data[random_index]

# Terminal data
maturecell_data = adata[adata.obs['segment'] == 3]
maturecell_data = maturecell_data[maturecell_data.obs['HPF'] == 12]
random_index = np.random.choice(np.arange(0,np.shape(maturecell_data)[0]),500)
maturecell_data = maturecell_data[random_index]

training_data = downsampled_proge_data.concatenate(maturecell_data)
training_data.obs["cells"] = "Training data"

/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [12]:
zebra_obj_78_3 = deep_predictor.DeepPredictor(training_data,latent_dim=100,hidden_layers=[800,800],workers=8,parameters=[0,0,1,0,1],likelihood="nb",batch_size=100)
zebra_obj_78_3.train(epochs=100)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type     | Params
-----------------------------------
0 | model | VAEmodel | 29.1 M
-----------------------------------
29.1 M    Trainable params
0         Non-trainable params
29.1 M    Total params
116.567   Total estimated model params size (MB)
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:1892: PossibleUserWarning: The number of training batches (10) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


In [14]:
#zebra_obj_78_3.save_model(path="/home/gaurav/Gaurav/Berlin/Paper/Models_single_vs_pool/Trained_on_mesoendoderm_predict_ectoderm/Single_type_78_3.pt")

In [15]:
# Starting data 
progenitors_data = adata[adata.obs['segment'] == 78]
progenitors_data = progenitors_data[progenitors_data.obs['HPF'] == 5.3]

# Downsampling Starting data
random_index = np.random.choice(np.arange(0,np.shape(progenitors_data)[0]),500)
downsampled_proge_data = progenitors_data[random_index]

# Terminal data
maturecell_data = adata[adata.obs['segment'] == 58]
maturecell_data = maturecell_data[maturecell_data.obs['HPF'] == 12]
random_index = np.random.choice(np.arange(0,np.shape(maturecell_data)[0]),500)
maturecell_data = maturecell_data[random_index]

training_data = downsampled_proge_data.concatenate(maturecell_data)
training_data.obs["cells"] = "Training data"

/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [16]:
zebra_obj_78_58 = deep_predictor.DeepPredictor(training_data,latent_dim=100,hidden_layers=[800,800],workers=8,parameters=[0,0,1,0,1],likelihood="nb",batch_size=100)
zebra_obj_78_58.train(epochs=100)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type     | Params
-----------------------------------
0 | model | VAEmodel | 29.1 M
-----------------------------------
29.1 M    Trainable params
0         Non-trainable params
29.1 M    Total params
116.567   Total estimated model params size (MB)
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:1892: PossibleUserWarning: The number of training batches (10) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


In [17]:
#zebra_obj_78_58.save_model(path="/home/gaurav/Gaurav/Berlin/Paper/Models_single_vs_pool/Trained_on_mesoendoderm_predict_ectoderm/Single_type_78_58.pt")

In [18]:
# Starting data 
progenitors_data = adata[adata.obs['segment'] == 78]
progenitors_data = progenitors_data[progenitors_data.obs['HPF'] == 5.3]

# Downsampling Starting data
random_index = np.random.choice(np.arange(0,np.shape(progenitors_data)[0]),500)
downsampled_proge_data = progenitors_data[random_index]

# Terminal data
maturecell_data = adata[adata.obs['segment'] == 5]
maturecell_data = maturecell_data[maturecell_data.obs['HPF'] == 12]
random_index = np.random.choice(np.arange(0,np.shape(maturecell_data)[0]),500)
maturecell_data = maturecell_data[random_index]

training_data = downsampled_proge_data.concatenate(maturecell_data)
training_data.obs["cells"] = "Training data"

/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [19]:
zebra_obj_78_5 = deep_predictor.DeepPredictor(training_data,latent_dim=100,hidden_layers=[800,800],workers=8,parameters=[0,0,1,0,1],likelihood="nb",batch_size=100)
zebra_obj_78_5.train(epochs=100)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type     | Params
-----------------------------------
0 | model | VAEmodel | 29.1 M
-----------------------------------
29.1 M    Trainable params
0         Non-trainable params
29.1 M    Total params
116.567   Total estimated model params size (MB)
/home/gaurav/anaconda3/envs/project/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:1892: PossibleUserWarning: The number of training batches (10) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


In [20]:
#zebra_obj_78_5.save_model(path="/home/gaurav/Gaurav/Berlin/Paper/Models_single_vs_pool/Trained_on_mesoendoderm_predict_ectoderm/Single_type_78_5.pt")